## Dependencies

In [1]:
# Import Dependencies
from tensorflow.keras.models import load_model
import joblib
import numpy as np
import pandas as pd
import requests

## Get Player Data

In [2]:
# Player data from URL
results = requests.get("http://nhl-app.eba-tmaqptju.us-west-2.elasticbeanstalk.com/api/avgplayerstats/all").json()

In [3]:
# Create Dataframe of player data
df = pd.DataFrame()
pid_list = []
pos_list = []
toi_list = []
g_list = []
a_list = []
s_list = []
h_list = []
ppg_list = []
ppa_list = []
pim_list = []
fow_list = []
fot_list = []
ta_list = []
ga_list = []
shg_list = []
sha_list = []
bs_list = []
etoi_list = []
shtoi_list = []
pptoi_list = []
for player in results:
    pid_list.append(player['player_id'])
    pos_list.append(player['Position'][-1])
    toi_list.append(player['timeOnIce'])
    a_list.append(player['assists'])
    g_list.append(player['goals'])
    s_list.append(player['shots'])
    h_list.append(player['hits'])
    ppg_list.append(player['powerPlayGoals'])
    ppa_list.append(player['powerPlayAssists'])
    pim_list.append(player['penaltyMinutes'])
    fow_list.append(player['faceOffWins'])
    fot_list.append(player['faceoffTaken'])
    ta_list.append(player['takeaways'])
    ga_list.append(player['Expr1'])
    shg_list.append(player['shortHandedGoals'])
    sha_list.append(player['shortHandedAssists'])
    bs_list.append(player['blocked'])
    etoi_list.append(player['evenTimeOnIce'])
    shtoi_list.append(player['shortHandedTimeOnIce'])
    pptoi_list.append(player['powerPlayTimeOnIce'])
df["pid"] = pid_list
df["pos"] = pos_list
df["toi"] = toi_list
df["g"] = g_list
df["a"] = g_list
df["s"] = s_list
df["h"] = h_list
df["ppg"] = ppg_list
df["ppa"] = ppa_list
df["pim"] = pim_list
df["fow"] = fow_list
df["fot"] = fot_list
df["ta"] = ta_list
df["ga"] = ga_list
df["shg"] = shg_list
df["sha"] = sha_list
df["bs"] = bs_list
df["etoi"] = etoi_list
df["shtoi"] = shtoi_list
df["pptoi"] = pptoi_list

In [4]:
X_df = df.drop(columns=["pos", "pid"])

In [5]:
# Establish X, y
X = X_df.values
y = df["pos"]

In [6]:
from sklearn.preprocessing import StandardScaler

# Standardize the columns
X_scaler = StandardScaler().fit(X)
X_scaled = X_scaler.transform(X)

## Import Models

In [7]:
# Import and save models
nn_model = load_model("position_nn_model.h5")
grid_model = joblib.load("position_grid_model.pkl")

## Neural Network

In [8]:
# Classification for the NN
classes = ["C", "D", "W"]

In [9]:
# Iterate through each row in the entire dataset
nn_predicted = []
for row in range(len(X_scaled)):
    test = np.expand_dims(X_scaled[row], axis=0)
    nn_predicted.append(classes[np.argmax(nn_model.predict(test), axis=-1)[0]])

In [10]:
# Add to df
df["nn_predicted"] = nn_predicted

In [11]:
# Add column to specify if the NN predicted the player's position correctly
df["nn_match"] = df["pos"] == df["nn_predicted"]

## Grid Search

In [12]:
# Use Grid Search Model to predict position
grid_pred_classes = grid_model.predict(X_scaled)

In [13]:
# Iterate through each predicted class and reclassify as the position
grid_predicted = [classes[i] for i in grid_pred_classes]

In [14]:
# Add to df
df["grid_predicted"] = grid_predicted

In [15]:
# Add column to specify if the Grid Search predicted the player's position correctly
df["grid_match"] = df["pos"] == df["grid_predicted"]

In [16]:
# Add column to specify if the Grid Search matched the NN
df["models_match"] = df["nn_predicted"] == df["grid_predicted"]

## Save Player Stats Table

In [17]:
new_data_df = df[["pid", "grid_predicted", "nn_predicted"]]

In [18]:
new_data_df

,pid,grid_predicted,nn_predicted
0,8478498,W,W
1,8471941,W,W
2,8479613,D,W
3,8473565,D,D
4,8471237,W,D
...,...,...,...
3348,8458805,D,D
3349,8467363,W,W
3350,8478506,D,D
3351,8466260,W,W


In [23]:
df.loc[df["nn_match"]==False,:].loc[df["models_match"]==False,:].loc[df["grid_match"]==False,:]

,pid,pos,nn_predicted,nn_match,grid_predicted,grid_match,models_match
420,8466174,C,D,False,W,False,False
436,8470671,C,D,False,W,False,False
597,8479511,C,W,False,D,False,False
729,8467700,C,D,False,W,False,False
757,8469522,C,D,False,W,False,False
935,8466191,C,D,False,W,False,False
1101,8473505,C,W,False,D,False,False
2386,8476339,C,W,False,D,False,False
2428,8480208,C,D,False,W,False,False
2516,8473491,C,D,False,W,False,False


In [19]:
# new_data_df.to_csv("predicted.csv")